In [68]:
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
from ha_nii_fit import Ha_NII
from fit_routine import lines

In [83]:
v1 = fits.open("../Ha_NII_fit/ha_nii_rebin_fit_mod2_2.fits")[13].data
v2 = fits.open("../Ha_NII_fit/ha_nii_rebin_fit_mod2_4.fits")[13].data
v3 = fits.open("../Ha_NII_fit/ha_nii_rebin_fit_mod2_5.fits")[13].data
v4 = fits.open("../Ha_NII_fit/ha_nii_rebin_fit_mod2_6.fits")[13].data

v1 = np.nan_to_num(v1, nan=1e6)
v2 = np.nan_to_num(v2, nan=1e6)
v3 = np.nan_to_num(v3, nan=1e6)
v4 = np.nan_to_num(v4, nan=1e6)
result = np.zeros(v1.shape)

for i in range(result.shape[0]):
    for j in range(result.shape[1]):
        chi1 = v1[i,j]
        chi2 = v2[i,j]
        chi3 = v3[i,j]
        chi4 = v4[i,j]
        minchi = min((chi1, chi2, chi3, chi4))
        if minchi == 1e6:
            result[i,j] = np.nan
        elif chi1 == minchi:
            result[i,j] = chi1
        elif chi2 == minchi:
            result[i,j] = chi2
        elif chi3 == minchi:
            result[i,j] = chi3
        elif chi4 == minchi:
            result[i,j] = chi4

resulthdu = fits.PrimaryHDU(result)
hdul = fits.HDUList([resulthdu])
hdul.writeto('../Ha_NII_fit/chi_map_val2.fits', overwrite=True)

In [94]:
hdul1 = fits.open("../cube_rebin.fits")
head = hdul1[0].header
cubehdu = hdul1[0]
cubehead = cubehdu.header
cube = cubehdu.data

hdu = fits.open("../varc_rebin.fits")
varc = hdu[0].data

hdul2 = fits.open("../basespec_smoothed.fits")
basespec = hdul2[0].data

fits1 = Ha_NII('Ha+NII', lines['Ha+NII'][0], lines['Ha+NII'][1], lines['Ha+NII'][2], cube, varc, basespec)
fits2 = Ha_NII('Ha+NII', lines['Ha+NII'][0], lines['Ha+NII'][1], lines['Ha+NII'][2], cube, varc, basespec)
fits3 = Ha_NII('Ha+NII', lines['Ha+NII'][0], lines['Ha+NII'][1], lines['Ha+NII'][2], cube, varc, basespec)
fits4 = Ha_NII('Ha+NII', lines['Ha+NII'][0], lines['Ha+NII'][1], lines['Ha+NII'][2], cube, varc, basespec)

fitresult =  Ha_NII('Ha+NII', lines['Ha+NII'][0], lines['Ha+NII'][1], lines['Ha+NII'][2], cube, varc, basespec)

fits1.load_fitcubes("../Ha_NII_fit/ha_nii_rebin_fit_mod2_2.fits")
fits2.load_fitcubes("../Ha_NII_fit/ha_nii_rebin_fit_mod2_4.fits")
fits3.load_fitcubes("../Ha_NII_fit/ha_nii_rebin_fit_mod2_5.fits")
fits4.load_fitcubes("../Ha_NII_fit/ha_nii_rebin_fit_mod2_6.fits")
result = np.zeros(v1.shape)

for i in range(v1.shape[0]):
    for j in range(v1.shape[1]):
        for f in (fits1,fits2,fits3, fits4):
            if any(f.fitcube[8:13, i, j] < 3):
                f.fitcube[13,i,j] == 1e6
        chi1 = fits1.fitcube[13,i,j]
        chi2 = fits2.fitcube[13,i,j]
        chi3 = fits3.fitcube[13,i,j]
        chi4 = fits4.fitcube[13,i,j]
        chil = np.nan_to_num([chi1, chi2, chi3, chi4], nan=1e6)
        minchi = min(chil)
        if i==40 and j==55:
            print(chil)
            print(fits2.fitcube[13, i,j])
            print(minchi)
        if minchi == 1e6:
            result[i,j] = np.nan
        elif chil[0] == minchi:
            result[i,j] = 1
        elif chil[1] == minchi:
            result[i,j] = 2
        elif chil[2] == minchi:
            result[i,j] = 3
        elif chil[3] == minchi:
            result[i,j] = 4
        if i==40 and j==55:
            print(result[i,j])   
            print(fits2.fitcube[:,i,j])
        if result[i,j] == 1.0:
            fitresult.fitcube[:,i,j] = fits1.fitcube[:,i,j]
            fitresult.fiterrcube[:,i,j] = fits1.fiterrcube[:,i,j]
        elif result[i,j] == 2.0:
            fitresult.fitcube[:,i,j] = fits2.fitcube[:,i,j]
            fitresult.fiterrcube[:,i,j] = fits2.fiterrcube[:,i,j]
        elif result[i,j] == 3.0:
            fitresult.fitcube[:,i,j] = fits3.fitcube[:,i,j]
            fitresult.fiterrcube[:,i,j] = fits3.fiterrcube[:,i,j]
        elif result[i,j] == 4.0:
            fitresult.fitcube[:,i,j] = fits4.fitcube[:,i,j]
            fitresult.fiterrcube[:,i,j] = fits4.fiterrcube[:,i,j]
        elif np.isnan(result[i,j]):
            fitresult.fitcube[:,i,j] = np.nan
            fitresult.fiterrcube[:,i,j] = np.nan
    
ha_nii = fitresult
newwcs = WCS(cubehead, naxis=2)
newhead = newwcs.to_header()
prihdu = fits.PrimaryHDU(ha_nii.fitcube[0], header=newhead)
newsiihdus = [fits.ImageHDU(ha_nii.fitcube[i])
                            for i in range(1, ha_nii.fitcube.shape[0])]
siierrhdus = [fits.ImageHDU(ha_nii.fiterrcube[i])
              for i in range(ha_nii.fiterrcube.shape[0])]
hdul = fits.HDUList([prihdu] + newsiihdus + siierrhdus)
hdul.writeto('../Ha_NII_fit/combine/ha_nii_rebin_fit_mod2_combine3.fits', overwrite=True)

rejhdus = fits.PrimaryHDU(ha_nii.rejcube[0], header=newhead)
otherrejhdus = [fits.ImageHDU(ha_nii.rejcube[i])
                              for i in range(1, ha_nii.rejcube.shape[0])]
hdul2 = fits.HDUList([rejhdus]+otherrejhdus)
hdul2.writeto('../Ha_NII_fit/combine/ha_nii_rebin_rej_mod2_combine3.fits', overwrite=True)

detectedimg = np.nan_to_num(ha_nii.fitcube[4])
snrrejimg = np.nan_to_num(ha_nii.rejcube[1])
evalimg = detectedimg + snrrejimg 

evalhdus = fits.PrimaryHDU(evalimg, header=newhead)
hdul3 = fits.HDUList([evalhdus])
hdul3.writeto('../Ha_NII_fit/combine/ha_nii_rebin_eval_mod2_combine3.fits', overwrite=True)

[7.19294801e-01 1.00000000e+06 4.13043374e+00 4.65989250e+00]
nan
0.7192948006341646
1.0
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan]


In [95]:
ha_nii = Ha_NII('Ha+NII', lines['Ha+NII'][0], lines['Ha+NII'][1], lines['Ha+NII'][2], cube, varc, basespec)
ha_nii.load_2fitcubes("../Ha_NII_fit/ha_nii_rebin_fit_mod1.fits", "../Ha_NII_fit/combine/ha_nii_rebin_fit_mod2_combine3.fits")

In [ ]:
PIXELS = [(44,40), (46,34),(55,36), (52,32), (70,41), (70,43), (74,41),(61,33)]
ha_nii.plot_eval(PIXELS)

In [ ]:
MOD1 = [(47,41), (51,44), (56,51), (63,47), (68,41), (67,32), (62,21), (43,30), (54,35), (58,39)]
ha_nii.plot_eval(MOD1, save=False, fname='Ha_NII_mod1_compare')

In [ ]:
HIGH_VEL = [(36,40), (39,39), (70,42), (66,51), (49,42), (52,43)]
ha_nii.plot_eval(HIGH_VEL, save=False, fname='Ha_NII_HIGH_VEL')


In [ ]:
FLUX_REJ = [(38,36), (49,42), (52,43), (52,42), (51,32), (56,34), (57,45), (55,45), (66,41), (57,33)]
ha_nii.plot_eval(FLUX_REJ)

In [ ]:
eval = fits.open("../Ha_NII_fit/select_2_4_comp.fits")[0].data
RATIO_EVAL = []
for i in range(eval.shape[0]):
    for j in range(eval.shape[1]):
        if eval[i,j] == -1:
            RATIO_EVAL.append((j+1, i+1))
ha_nii.plot_eval(RATIO_EVAL)

In [96]:
# def select(i, j):
#     self = ha_nii
#     self.fitcube = np.zeros((14, self.cube_x, self.cube_y))
#     self.fiterrcube = np.zeros((8 ,self.cube_x, self.cube_y))
    
#     popt1 = self.mod1_fitcube[4:8, i, j]
#     snr_nii = self.mod1_fitcube[8, i, j]
#     snr_ha = self.mod1_fitcube[9, i, j]
#     chi1 = self.mod1_fitcube[10, i, j]

#     popt2 = self.mod2_fitcube[0:8, i, j]
#     snr_nii_1 = self.mod2_fitcube[8, i, j].copy()
#     snr_ha_1 = self.mod2_fitcube[9, i, j].copy()
#     snr_nii_2 = self.mod2_fitcube[10, i, j].copy()
#     snr_ha_2 = self.mod2_fitcube[11, i, j].copy()
#     vel_snr = self.mod2_fitcube[12, i, j]
#     chi2 = self.mod2_fitcube[13, i, j]
#     nii_ratio = popt2[0]/popt2[4] if popt2[0] < popt2[4] else popt2[4]/popt2[0]
#     ha_ratio = popt2[1]/popt2[5] if popt2[1] < popt2[5] else popt2[5]/popt2[1]

#     # Choose mod1 if well-fitted
#     if snr_nii > 3. and snr_ha > 3. and chi1 <= 1.:
#         self.mod2_fitcube[:,i,j] = np.nan
#         self.mod2_fiterrcube[:,i,j] = np.nan
#         return 1
#     # Choose mod2 if the difference is significant
#     if snr_nii_1 > 3. and snr_ha_1 > 3. and snr_nii_2 > 3. and snr_ha_2 > 3. and vel_snr > 3. and chi2 < chi1 and nii_ratio > 0.05 and ha_ratio > 0.05:

#         self.mod1_fitcube[:,i,j] = np.nan
#         self.mod1_fiterrcube[:,i,j] = np.nan 

#         # Component left/right check
#         if self.mod2_fitcube[0,i,j] + self.mod2_fitcube[1,i,j] > self.mod2_fitcube[4,i,j] + self.mod2_fitcube[5,i,j]:
#             tmp = self.mod2_fitcube[0:4, i, j].copy()
#             self.mod2_fitcube[0:4, i, j] = self.mod2_fitcube[4:8, i, j].copy()
#             self.mod2_fitcube[4:8, i, j] = tmp.copy()
#             tmp = self.mod2_fiterrcube[0:4, i, j].copy()
#             self.mod2_fiterrcube[0:4, i, j] = self.mod2_fitcube[4:8, i, j].copy()
#             self.mod2_fiterrcube[4:8, i, j] = tmp.copy()
#             self.mod2_fitcube[8, i, j] = snr_nii_2
#             self.mod2_fitcube[9, i, j] = snr_ha_2
#             self.mod2_fitcube[10, i, j] = snr_nii_1
#             self.mod2_fitcube[11, i, j] = snr_ha_1
            

#         return 2
#     elif snr_nii > 3. and snr_ha > 3.:
#         self.mod2_fitcube[:,i,j] = np.nan
#         self.mod2_fiterrcube[:,i,j] = np.nan
#         return 1
#     else:
#         self.mod1_fitcube[:,i,j] = np.nan
#         self.mod1_fiterrcube[:,i,j] = np.nan 
#         self.mod2_fitcube[:,i,j] = np.nan
#         self.mod2_fiterrcube[:,i,j] = np.nan
#         return 0
#     ...

select_img = np.full((ha_nii.cube_x, ha_nii.cube_y), np.nan)
for i in range(ha_nii.cube_x):
    for j in range(ha_nii.cube_y):
       select_img[i,j] = ha_nii.select(i,j)

#Save the 2 cubes
newwcs = WCS(cubehead, naxis=2)
newhead = newwcs.to_header()
prihdu = fits.PrimaryHDU(ha_nii.mod1_fitcube[0], header=newhead)
newsiihdus = [fits.ImageHDU(ha_nii.mod1_fitcube[i])
                            for i in range(1, ha_nii.mod1_fitcube.shape[0])]
siierrhdus = [fits.ImageHDU(ha_nii.mod1_fiterrcube[i])
              for i in range(ha_nii.mod1_fiterrcube.shape[0])]
hdul = fits.HDUList([prihdu] + newsiihdus + siierrhdus)
hdul.writeto('../Ha_NII_fit/combine/ha_nii_rebin_fit_mod1_select_vel_v3.fits', overwrite=True)

prihdu = fits.PrimaryHDU(ha_nii.mod2_fitcube[0], header=newhead)
newsiihdus = [fits.ImageHDU(ha_nii.mod2_fitcube[i])
                            for i in range(1, ha_nii.mod2_fitcube.shape[0])]
siierrhdus = [fits.ImageHDU(ha_nii.mod2_fiterrcube[i])
              for i in range(ha_nii.mod2_fiterrcube.shape[0])]
hdul = fits.HDUList([prihdu] + newsiihdus + siierrhdus)
hdul.writeto('../Ha_NII_fit/combine/ha_nii_rebin_fit_mod2_2_select_vel_v3.fits', overwrite=True)


evalhdus = fits.PrimaryHDU(select_img, header=newhead)
hdul3 = fits.HDUList([evalhdus])
hdul3.writeto('../Ha_NII_fit/combine/ha_nii_rebin_select_vel_v3.fits', overwrite=True)